##IMPORT LIBRARY

In [1]:
#import surprise
import pandas as pd
import numpy as np
#from surprise.model_selection import GridSearchCV
from scipy import sparse

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt

## IMPORT DATA

In [2]:
mydata = pd.read_csv('Data Orbit - Data Gabungan.csv')
mydata.head()

,ID_Tempat,Nama_Tempat,ID_User,Nama_User,Daerah,Latitude,Longitude,Rating,Telepon
0,t0001,"Babi Guling & Siobak Singaraja ""Sandi""",u0035,agus upinn,Denpasar Utara,"-8,636,253","115,193,073",5,85965900360
1,t0001,"Babi Guling & Siobak Singaraja ""Sandi""",u0057,anandha putra i gusti putu,Denpasar Utara,"-8,636,253","115,193,073",5,85965900360
2,t0001,"Babi Guling & Siobak Singaraja ""Sandi""",u0293,gede restu,Denpasar Utara,"-8,636,253","115,193,073",5,85965900360
3,t0001,"Babi Guling & Siobak Singaraja ""Sandi""",u0297,GEDE SUMPENA,Denpasar Utara,"-8,636,253","115,193,073",2,85965900360
4,t0001,"Babi Guling & Siobak Singaraja ""Sandi""",u0349,I KADEK WIRAWAN,Denpasar Utara,"-8,636,253","115,193,073",5,85965900360


In [3]:
mydata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 813 entries, 0 to 812
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID_Tempat    813 non-null    object
 1   Nama_Tempat  813 non-null    object
 2   ID_User      813 non-null    object
 3   Nama_User    813 non-null    object
 4   Daerah       813 non-null    object
 5   Latitude     813 non-null    object
 6   Longitude    813 non-null    object
 7   Rating       813 non-null    int64 
 8   Telepon      813 non-null    object
dtypes: int64(1), object(8)
memory usage: 57.3+ KB


In [4]:
datamy = pd.read_csv('Data Orbit - Data Tempat lengkap.csv')
datamy.head()

,ID_Tempat,Nama_Tempat,Daerah,Latitude,Longitude,Telepon,Gambar,Menu
0,t0001,"Babi Guling & Siobak Singaraja ""Sandi""",Denpasar Utara,"-8,636,253","115,193,073",85965900360,"1_a,1_b","Nasi Babi Guling Biasa, Nasi Babi Guling Spesi..."
1,t0002,Wr. Babi Guling Mawar,Denpasar Utara,"-8,627,955","115,208,472",81805313408,"2_a,2_b,2_c","Nasi Babi Guling Siram Bumbu, Nasi Be Guling, ..."
2,t0003,Wr. Makan Pak Joni,Denpasar Selatan,"-87,027,762","1,152,305,969",85237101606,"3_a,3_b,3_c,3_d","Nasi Lawar Sapi, Nasi Lawar, Nasi Lawar Kompli..."
3,t0004,Wr. Babi Guling Cahaya Bintang Tina,Denpasar Utara,"-8,636,147","115,210,471",81236381964,"4_a,4_b,4_c","Nasi Babi Guling Istimewa, Nasi Babi Guling Bi..."
4,t0005,Wr. Babi Guling Sandi,Denpasar Utara,"-8,615,789","115,212,484",81999588028,5_a,"Nasi Babi Guling Komplit, Babi Guling Spesial,..."


In [5]:
datamy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID_Tempat    100 non-null    object
 1   Nama_Tempat  100 non-null    object
 2   Daerah       100 non-null    object
 3   Latitude     100 non-null    object
 4   Longitude    100 non-null    object
 5   Telepon      100 non-null    object
 6   Gambar       100 non-null    object
 7   Menu         100 non-null    object
dtypes: object(8)
memory usage: 6.4+ KB


## PREPROCESSING

In [6]:
mydata['Rating'] = mydata['Rating'].replace(r'\D+','',regex=True)
mydata['Rating'] = pd.to_numeric(mydata.Rating)

In [7]:
mydata['Latitude'] = mydata['Latitude'].replace(r'\D+','',regex=True)
mydata['Latitude'] = pd.to_numeric(mydata.Rating)
mydata['Longitude'] = mydata['Longitude'].replace(r'\D+','',regex=True)
mydata['Longitude'] = pd.to_numeric(mydata.Rating)

In [8]:
mydata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 813 entries, 0 to 812
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID_Tempat    813 non-null    object
 1   Nama_Tempat  813 non-null    object
 2   ID_User      813 non-null    object
 3   Nama_User    813 non-null    object
 4   Daerah       813 non-null    object
 5   Latitude     813 non-null    int64 
 6   Longitude    813 non-null    int64 
 7   Rating       813 non-null    int64 
 8   Telepon      813 non-null    object
dtypes: int64(3), object(6)
memory usage: 57.3+ KB


In [9]:
mydata.isnull().sum()

ID_Tempat      0
Nama_Tempat    0
ID_User        0
Nama_User      0
Daerah         0
Latitude       0
Longitude      0
Rating         0
Telepon        0
dtype: int64

## Membuat dictionary tempat

In [10]:
tempat_new = pd.DataFrame({
    'id': datamy.ID_Tempat,
    'nama': datamy.Nama_Tempat,
    'lokasi': datamy.Daerah,
    'gambar': datamy.Gambar,
    'menu': datamy.Menu
})
tempat_new

,id,nama,lokasi,gambar,menu
0,t0001,"Babi Guling & Siobak Singaraja ""Sandi""",Denpasar Utara,"1_a,1_b","Nasi Babi Guling Biasa, Nasi Babi Guling Spesi..."
1,t0002,Wr. Babi Guling Mawar,Denpasar Utara,"2_a,2_b,2_c","Nasi Babi Guling Siram Bumbu, Nasi Be Guling, ..."
2,t0003,Wr. Makan Pak Joni,Denpasar Selatan,"3_a,3_b,3_c,3_d","Nasi Lawar Sapi, Nasi Lawar, Nasi Lawar Kompli..."
3,t0004,Wr. Babi Guling Cahaya Bintang Tina,Denpasar Utara,"4_a,4_b,4_c","Nasi Babi Guling Istimewa, Nasi Babi Guling Bi..."
4,t0005,Wr. Babi Guling Sandi,Denpasar Utara,5_a,"Nasi Babi Guling Komplit, Babi Guling Spesial,..."
...,...,...,...,...,...
95,t0096,Warung Makan Babi Guling Bu Manis,Denpasar,"96_a,96_b","Nasi Babi Guling Pisah, Nasi Babi Guling Campu..."
96,t0097,Rumah Makan Babi Guling Sari Nadi,Denpasar,"97_a,97_b","Nasi Babi Guling Spesial, Nasi Babi Guling Bia..."
97,t0098,Lawar Kuwir Yan Jinah,Denpasar,"98_a,98_b","Nasi Campur Lawar Kuwir Yan Jinah, Lawar Kuwir..."
98,t0099,Warung Lawar Khana,Denpasar,"99_a,99_b","Nasi Lawar Campur, Nasi Soto Lawar Campur, Nas..."


## Data preparation
Melakukan persiapan data untuk menyandikan (encode) fitur 'ID_User' dan 'ID_Tempat' ke dalam indeks integer.

In [11]:
# Mengubah ID_User menjadi list tanpa nilai yang sama
user_ids = mydata['ID_User'].unique().tolist()
print('list ID_User: ', user_ids)
 
# Melakukan encoding ID_User
user_to_user_encoded = {x: i for i, x in enumerate(user_ids)}
print('encoded ID_User : ', user_to_user_encoded)
 
# Melakukan proses encoding angka ke ke ID_User
user_encoded_to_user = {i: x for i, x in enumerate(user_ids)}
print('encoded angka ke ID_User: ', user_encoded_to_user)

list ID_User:  ['u0035', 'u0057', 'u0293', 'u0297', 'u0349', 'u0356', 'u0531', 'u0542', 'u0640', 'u0641', 'u0016', 'u0027', 'u0060', 'u0088', 'u0288', 'u0355', 'u0514', 'u0516', 'u0617', 'u0626', 'u0028', 'u0137', 'u0335', 'u0338', 'u0360', 'u0364', 'u0467', 'u0684', 'u0701', 'u0758', 'u0026', 'u0051', 'u0133', 'u0186', 'u0251', 'u0292', 'u0379', 'u0449', 'u0721', 'u0723', 'u0090', 'u0197', 'u0223', 'u0320', 'u0439', 'u0456', 'u0557', 'u0568', 'u0673', 'u0145', 'u0230', 'u0327', 'u0383', 'u0480', 'u0643', 'u0675', 'u0686', 'u0698', 'u0010', 'u0152', 'u0182', 'u0333', 'u0345', 'u0359', 'u0437', 'u0481', 'u0484', 'u0548', 'u0032', 'u0233', 'u0258', 'u0272', 'u0390', 'u0401', 'u0475', 'u0573', 'u0720', 'u0760', 'u0021', 'u0046', 'u0054', 'u0119', 'u0422', 'u0490', 'u0533', 'u0581', 'u0682', 'u0716', 'u0059', 'u0154', 'u0195', 'u0289', 'u0386', 'u0463', 'u0486', 'u0559', 'u0616', 'u0773', 'u0151', 'u0185', 'u0189', 'u0229', 'u0332', 'u0380', 'u0419', 'u0457', 'u0576', 'u0623', 'u0131', 'u0

In [12]:
# Mengubah ID_Tempat menjadi list tanpa nilai yang sama
tempat_ids = mydata['ID_Tempat'].unique().tolist()
 
# Melakukan proses encoding ID_Tempat
tempat_to_tempat_encoded = {x: i for i, x in enumerate(tempat_ids)}
 
# Melakukan proses encoding angka ke ID_Tempat
tempat_encoded_to_tempat = {i: x for i, x in enumerate(tempat_ids)}
 
# Selanjutnya, petakan ID_User dan ID_Tempat ke dataframe yang berkaitan.
 
# Mapping ID_User ke dataframe user
mydata['Nama_User'] = mydata['ID_User'].map(user_to_user_encoded)
 
# Mapping ID_Tempat ke dataframe tempat
mydata['Nama_Tempat'] = mydata['ID_Tempat'].map(tempat_to_tempat_encoded)

In [13]:
# Mendapatkan jumlah user
num_users = len(user_to_user_encoded)
print(num_users)
 
# Mendapatkan jumlah tempat
num_tempat = len(tempat_encoded_to_tempat)
print(num_tempat)
 
# Mengubah rating menjadi nilai float
mydata['Rating'] = mydata['Rating'].values.astype(np.float32)
 
# Nilai minimum rating
min_rating = min(mydata['Rating'])
 
# Nilai maksimal rating
max_rating = max(mydata['Rating'])
 
print('Number of User: {}, Number of Tempat: {}, Min Rating: {}, Max Rating: {}'.format(
    num_users, num_tempat, min_rating, max_rating
))

781
100
Number of User: 781, Number of Tempat: 100, Min Rating: 1.0, Max Rating: 5.0


### Membagi Data Untuk Data Training dan Validasi

In [14]:
# Mengacak dataset
mydata = mydata.sample(frac=1, random_state=42)
mydata

,ID_Tempat,Nama_Tempat,ID_User,Nama_User,Daerah,Latitude,Longitude,Rating,Telepon
247,t0031,30,u0164,244,Denpsasar Timur,5,5,5.0,83117263509
589,t0073,72,u0007,569,"Badung, Kuta",5,5,5.0,81236288555
227,t0028,27,u0234,224,Denpasar Timur,5,5,5.0,(0361) 34130
291,t0038,37,u0361,285,Denpasar Timur,5,5,5.0,85396939662
539,t0067,66,u0562,524,Denpasar,5,5,5.0,85847417989
...,...,...,...,...,...,...,...,...,...
71,t0008,7,u0233,69,Denpasar Selatan,5,5,5.0,87861187505
106,t0011,10,u0419,104,Denpasar Selatan,5,5,5.0,81236155803
270,t0034,33,u0375,265,Denpasar Timur,5,5,5.0,81999383447
435,t0053,52,u0663,425,Denpasar,5,5,5.0,811392791


In [15]:
# Membuat variabel x untuk mencocokkan data nama user dan nama tempat menjadi satu value
x = mydata[['Nama_User', 'Nama_Tempat']].values
 
# Membuat variabel y untuk membuat rating dari hasil 
y = mydata['Rating'].apply(lambda x: (x - min_rating) / (max_rating - min_rating)).values
 
# Membagi menjadi 80% data train dan 20% data validasi
train_indices = int(0.8 * mydata.shape[0])
x_train, x_val, y_train, y_val = (
    x[:train_indices],
    x[train_indices:],
    y[:train_indices],
    y[train_indices:]
)
 
print(x, y)

[[244  30]
 [569  72]
 [224  27]
 ...
 [265  33]
 [425  52]
 [100  10]] [1.   1.   1.   1.   1.   0.75 1.   0.75 1.   0.75 1.   1.   1.   0.75
 1.   0.   0.75 1.   1.   1.   1.   1.   1.   0.75 1.   1.   1.   1.
 1.   0.   1.   1.   1.   1.   1.   0.   1.   1.   1.   1.   1.   1.
 0.75 1.   0.   1.   1.   1.   1.   1.   0.75 1.   0.75 1.   1.   1.
 0.5  1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   0.5
 1.   0.75 1.   1.   0.5  0.75 1.   0.75 0.25 0.75 1.   1.   1.   1.
 1.   0.75 1.   0.5  1.   0.75 1.   1.   0.5  0.5  1.   0.75 1.   1.
 1.   1.   0.75 1.   1.   1.   1.   0.75 0.   1.   1.   1.   1.   0.
 1.   1.   1.   1.   1.   0.   0.   1.   1.   1.   1.   1.   0.75 0.5
 1.   1.   1.   1.   1.   0.5  0.5  0.75 0.75 0.   0.75 1.   1.   1.
 1.   1.   1.   1.   1.   1.   1.   1.   1.   0.75 1.   1.   1.   1.
 1.   1.   1.   1.   1.   1.   1.   1.   0.75 1.   1.   1.   1.   1.
 1.   0.75 0.75 1.   1.   0.5  1.   0.75 1.   1.   0.75 0.75 1.   1.
 1.   0.75 1.   0.75 1.   1

### Proses Training

In [16]:
class RecommenderNet(tf.keras.Model):
  
  # Insialisasi fungsi
  def __init__(self, num_users, num_tempat, embedding_size, **kwargs):
    super(RecommenderNet, self).__init__(**kwargs)
    self.num_users = num_users
    self.num_tempat = num_tempat
    self.embedding_size = embedding_size
    self.user_embedding = layers.Embedding( # layer embedding user
        num_users,
        embedding_size,
        embeddings_initializer = 'he_normal',
        embeddings_regularizer = keras.regularizers.l2(1e-6)
    )
    self.user_bias = layers.Embedding(num_users, 1) # layer embedding user bias
    self.tempat_embedding = layers.Embedding( # layer embeddings tempat
        num_tempat,
        embedding_size,
        embeddings_initializer = 'he_normal',
        embeddings_regularizer = keras.regularizers.l2(1e-6)
    )
    self.tempat_bias = layers.Embedding(num_tempat, 1) # layer embedding tempat bias
 
  def call(self, inputs):
    user_vector = self.user_embedding(inputs[:,0]) # memanggil layer embedding 1
    user_bias = self.user_bias(inputs[:, 0]) # memanggil layer embedding 2
    tempat_vector = self.tempat_embedding(inputs[:, 1]) # memanggil layer embedding 3
    tempat_bias = self.tempat_bias(inputs[:, 1]) # memanggil layer embedding 4
 
    dot_user_tempat = tf.tensordot(user_vector, tempat_vector, 2) 
 
    x = dot_user_tempat + user_bias + tempat_bias
    
    return tf.nn.sigmoid(x) # activation sigmoid

In [17]:
model = RecommenderNet(num_users, num_tempat, 50)
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

##Yang model disave terus yg hasil rekomendasi tu di python webnya



In [18]:
history = model.fit(
    x = x_train,
    y = y_train,
    batch_size = 8,
    epochs = 100,
    validation_data = (x_val, y_val)
)

Epoch 1/100
82/82 [==============================] - 1s 3ms/step - loss: 0.6808 - root_mean_squared_error: 0.4334 - val_loss: 0.6841 - val_root_mean_squared_error: 0.4299
Epoch 2/100
82/82 [==============================] - 0s 1ms/step - loss: 0.5613 - root_mean_squared_error: 0.3595 - val_loss: 0.6786 - val_root_mean_squared_error: 0.4267
Epoch 3/100
82/82 [==============================] - 0s 2ms/step - loss: 0.4832 - root_mean_squared_error: 0.3063 - val_loss: 0.6744 - val_root_mean_squared_error: 0.4242
Epoch 4/100
82/82 [==============================] - 0s 1ms/step - loss: 0.4403 - root_mean_squared_error: 0.2772 - val_loss: 0.6715 - val_root_mean_squared_error: 0.4225
Epoch 5/100
82/82 [==============================] - 0s 1ms/step - loss: 0.4143 - root_mean_squared_error: 0.2605 - val_loss: 0.6698 - val_root_mean_squared_error: 0.4215
Epoch 6/100
82/82 [==============================] - 0s 1ms/step - loss: 0.4066 - root_mean_squared_error: 0.2567 - val_loss: 0.6693 - val_root_m

Epoch 49/100
82/82 [==============================] - 0s 3ms/step - loss: 0.3097 - root_mean_squared_error: 0.1957 - val_loss: 0.6370 - val_root_mean_squared_error: 0.4022
Epoch 50/100
82/82 [==============================] - 0s 2ms/step - loss: 0.3132 - root_mean_squared_error: 0.1986 - val_loss: 0.6363 - val_root_mean_squared_error: 0.4018
Epoch 51/100
82/82 [==============================] - 0s 2ms/step - loss: 0.3111 - root_mean_squared_error: 0.1955 - val_loss: 0.6356 - val_root_mean_squared_error: 0.4014
Epoch 52/100
82/82 [==============================] - 0s 2ms/step - loss: 0.3092 - root_mean_squared_error: 0.1950 - val_loss: 0.6344 - val_root_mean_squared_error: 0.4008
Epoch 53/100
82/82 [==============================] - 0s 2ms/step - loss: 0.3136 - root_mean_squared_error: 0.1982 - val_loss: 0.6345 - val_root_mean_squared_error: 0.4008
Epoch 54/100
82/82 [==============================] - 0s 2ms/step - loss: 0.3134 - root_mean_squared_error: 0.1994 - val_loss: 0.6342 - val_

82/82 [==============================] - 0s 2ms/step - loss: 0.2678 - root_mean_squared_error: 0.1589 - val_loss: 0.6154 - val_root_mean_squared_error: 0.3905
Epoch 97/100
82/82 [==============================] - 0s 1ms/step - loss: 0.2683 - root_mean_squared_error: 0.1573 - val_loss: 0.6148 - val_root_mean_squared_error: 0.3902
Epoch 98/100
82/82 [==============================] - 0s 1ms/step - loss: 0.2638 - root_mean_squared_error: 0.1516 - val_loss: 0.6147 - val_root_mean_squared_error: 0.3902
Epoch 99/100
82/82 [==============================] - 0s 1ms/step - loss: 0.2661 - root_mean_squared_error: 0.1557 - val_loss: 0.6141 - val_root_mean_squared_error: 0.3899
Epoch 100/100
82/82 [==============================] - 0s 1ms/step - loss: 0.2643 - root_mean_squared_error: 0.1540 - val_loss: 0.6138 - val_root_mean_squared_error: 0.3897


In [19]:
tempat_mydata = tempat_new
data = pd.read_csv('Data Orbit - Data Gabungan.csv')
mydata = data[1:814]

# Mengambil sample user
user_id = mydata['ID_User'].sample(100).iloc[30]
tempat_visited_by_user = mydata[mydata['ID_User'] == user_id]
 
# Operator bitwise (~)
tempat_not_visited = tempat_mydata[~tempat_mydata['id'].isin(tempat_visited_by_user.ID_Tempat.values)]['id'] 
tempat_not_visited = list(
    set(tempat_not_visited)
    .intersection(set(tempat_to_tempat_encoded.keys()))
)
 
tempat_not_visited = [[tempat_to_tempat_encoded.get(x)] for x in tempat_not_visited]
user_encoder = user_to_user_encoded.get(user_id)
user_tempat_array = np.hstack(
    ([[user_encoder]] * len(tempat_not_visited), tempat_not_visited)
)

In [20]:
model.save('./saved_models/my_tf_model')

INFO:tensorflow:Assets written to: ./saved_models/my_tf_model\assets


In [21]:
my_tf_saved_model = tf.keras.models.load_model('./saved_models/my_tf_model')
my_tf_saved_model.summary()

Model: "recommender_net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  39050     
                                                                 
 embedding_1 (Embedding)     multiple                  781       
                                                                 
 embedding_2 (Embedding)     multiple                  5000      
                                                                 
 embedding_3 (Embedding)     multiple                  100       
                                                                 
Total params: 44,931
Trainable params: 44,931
Non-trainable params: 0
_________________________________________________________________


In [49]:
history = my_tf_saved_model.fit(
    x = x_train,
    y = y_train,
    batch_size = 8,
    epochs = 100,
    validation_data = (x_val, y_val)
)

Epoch 1/100
82/82 [==============================] - 0s 3ms/step - loss: 0.2104 - root_mean_squared_error: 0.0900 - val_loss: 0.5870 - val_root_mean_squared_error: 0.3763
Epoch 2/100
82/82 [==============================] - 0s 3ms/step - loss: 0.2113 - root_mean_squared_error: 0.0920 - val_loss: 0.5871 - val_root_mean_squared_error: 0.3763
Epoch 3/100
82/82 [==============================] - 0s 3ms/step - loss: 0.2122 - root_mean_squared_error: 0.0939 - val_loss: 0.5868 - val_root_mean_squared_error: 0.3761
Epoch 4/100
82/82 [==============================] - 0s 2ms/step - loss: 0.2109 - root_mean_squared_error: 0.0901 - val_loss: 0.5867 - val_root_mean_squared_error: 0.3761
Epoch 5/100
82/82 [==============================] - 0s 3ms/step - loss: 0.2094 - root_mean_squared_error: 0.0892 - val_loss: 0.5867 - val_root_mean_squared_error: 0.3761
Epoch 6/100
82/82 [==============================] - 0s 3ms/step - loss: 0.2085 - root_mean_squared_error: 0.0887 - val_loss: 0.5860 - val_root_m

Epoch 49/100
82/82 [==============================] - 0s 2ms/step - loss: 0.1967 - root_mean_squared_error: 0.0694 - val_loss: 0.5799 - val_root_mean_squared_error: 0.3723
Epoch 50/100
82/82 [==============================] - 0s 2ms/step - loss: 0.1970 - root_mean_squared_error: 0.0702 - val_loss: 0.5798 - val_root_mean_squared_error: 0.3723
Epoch 51/100
82/82 [==============================] - 0s 2ms/step - loss: 0.1958 - root_mean_squared_error: 0.0663 - val_loss: 0.5797 - val_root_mean_squared_error: 0.3722
Epoch 52/100
82/82 [==============================] - 0s 2ms/step - loss: 0.1963 - root_mean_squared_error: 0.0693 - val_loss: 0.5798 - val_root_mean_squared_error: 0.3723
Epoch 53/100
82/82 [==============================] - 0s 2ms/step - loss: 0.1966 - root_mean_squared_error: 0.0693 - val_loss: 0.5792 - val_root_mean_squared_error: 0.3720
Epoch 54/100
82/82 [==============================] - 0s 3ms/step - loss: 0.1951 - root_mean_squared_error: 0.0658 - val_loss: 0.5791 - val_

82/82 [==============================] - 0s 2ms/step - loss: 0.1875 - root_mean_squared_error: 0.0551 - val_loss: 0.5755 - val_root_mean_squared_error: 0.3696
Epoch 97/100
82/82 [==============================] - 0s 2ms/step - loss: 0.1871 - root_mean_squared_error: 0.0541 - val_loss: 0.5753 - val_root_mean_squared_error: 0.3695
Epoch 98/100
82/82 [==============================] - 0s 2ms/step - loss: 0.1869 - root_mean_squared_error: 0.0538 - val_loss: 0.5748 - val_root_mean_squared_error: 0.3692
Epoch 99/100
82/82 [==============================] - 0s 2ms/step - loss: 0.1867 - root_mean_squared_error: 0.0532 - val_loss: 0.5747 - val_root_mean_squared_error: 0.3692
Epoch 100/100
82/82 [==============================] - 0s 2ms/step - loss: 0.1865 - root_mean_squared_error: 0.0510 - val_loss: 0.5745 - val_root_mean_squared_error: 0.3691


In [50]:
rating = model.predict(user_tempat_array).flatten()
 
top_rating_indices = rating.argsort()
recommended_tempat_ids = [
    tempat_encoded_to_tempat.get(tempat_not_visited[x][0]) for x in top_rating_indices
]
 
print('Menampilkan Rekomendasi  Tempat Makan dan Lokasi untuk Pengguna dengan Total : {} Pengguna'.format(user_id))
print('===' * 11)
top_tempat_user = (
    tempat_visited_by_user.sort_values(
        by = 'Rating',
        ascending=False
    )
    .head(5)
    .ID_Tempat.values
)
 
tempat_mydata_rows = tempat_mydata[tempat_mydata['id'].isin(top_tempat_user)]
for row in tempat_mydata_rows.itertuples():
    print(row.nama, ':', row.lokasi, ':', row.gambar, ':', row.menu)
 
print('---' * 11)
print('10 Rekomendasi tempat makan')
print('---' * 11)
 
recommended_tempat = tempat_mydata[tempat_mydata['id'].isin(recommended_tempat_ids)]
for row in recommended_tempat.itertuples():
    print(row.nama, ':', row.lokasi,':', row.gambar, ':', row.menu)

Menampilkan Rekomendasi  Tempat Makan dan Lokasi untuk Pengguna dengan Total : u0486 Pengguna
Warung Adi : Denpasar Selatan : 10_a,10_b,10_c,10_d,10_e : Nasi Campur Porsi Biasa, Nasi Campur Porsi Sedang, Nasi Campur Porsi Besar, Nasi Campur Porsi Spesial, Nasi Campur Porsi Komplit
---------------------------------
10 Rekomendasi tempat makan
---------------------------------
Babi Guling & Siobak Singaraja "Sandi" : Denpasar Utara : 1_a,1_b : Nasi Babi Guling Biasa, Nasi Babi Guling Spesial, Lawar Putih, Lawar Merah, Bakso Babi
Wr. Babi Guling Mawar : Denpasar Utara : 2_a,2_b,2_c : Nasi Babi Guling Siram Bumbu, Nasi Be Guling, Paket Nasi Be Guling, Nasi Babi Guling Pahe, Nasi Be Guling Dobel Kulit
Wr. Makan Pak Joni : Denpasar Selatan : 3_a,3_b,3_c,3_d : Nasi Lawar Sapi, Nasi Lawar, Nasi Lawar Komplit, Nasi Soto, Soto Sapi
Wr. Babi Guling Cahaya Bintang Tina : Denpasar Utara : 4_a,4_b,4_c : Nasi Babi Guling Istimewa, Nasi Babi Guling Biasa, Babi Guling Spesial, Kuah Balung, Nasi Lawar
W

### Evaluasi

In [24]:
from sklearn.metrics import mean_squared_error
import math
result = mean_squared_error(y_true=y_val, y_pred=model.predict(x_val))/1e3
RMSE = math.sqrt(result)
print("RMSE dari data = ", RMSE) 

RMSE dari data =  0.012326506804227956


Hasil evaluasi yakni nilai RMSE adalah 0,0126 yang berarti  semakin mendekati 0 semakin akurat

### Kode Program Yang bukan rekomendasi Coach

In [25]:
!pip install scikit-surprise

  Running setup.py clean for scikit-surprise
Failed to build scikit-surprise
    Running setup.py install for scikit-surprise: started
    Running setup.py install for scikit-surprise: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\Ayu Triana\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Ayu Triana\\AppData\\Local\\Temp\\pip-install-4_u313sy\\scikit-surprise_b25b11ab464b44c7ba83933f0ba928fc\\setup.py'"'"'; __file__='"'"'C:\\Users\\Ayu Triana\\AppData\\Local\\Temp\\pip-install-4_u313sy\\scikit-surprise_b25b11ab464b44c7ba83933f0ba928fc\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Ayu Triana\AppData\Local\Temp\pip-wheel-2ggqgv40'
       cwd: C:\Users\Ayu Triana\AppData\Local\Temp\pip-install-4_u313sy\scikit-surprise_b25b11ab464b44c7ba83933f0ba928fc\
  Complete output (52 lines):
  C:\Users\Ayu Triana\anaconda3\lib\site-packages\setuptools\dist

In [26]:
mydata = mydata.drop(columns=['Nama_Tempat', 'Nama_User', 'Daerah', 'Latitude', 'Longitude', 'Telepon'], axis=1)
mydata

,ID_Tempat,ID_User,Rating
1,t0001,u0057,5
2,t0001,u0293,5
3,t0001,u0297,2
4,t0001,u0349,5
5,t0001,u0356,5
...,...,...,...
808,t0100,u0220,5
809,t0100,u0434,4
810,t0100,u0435,4
811,t0100,u0635,5


In [27]:
# main dataframe to be used
df = mydata.loc[:, ['ID_User', 'ID_Tempat', 'Rating']]

In [28]:
mydata.ID_User.nunique()

780

In [29]:
mydata.ID_Tempat.nunique()

100

## CONTINOUS ID MAPPING

In [30]:
def proc_col(col):
    uniq = col.unique()
    name2idx = {o: i for i, o in enumerate(uniq)}
    idx2name = {i: e for i, e in enumerate(name2idx.keys())}
    return idx2name, np.array([name2idx[x] for x in col]), len(uniq)

In [31]:
def encode_data(df):
    idx2user, user_col, num_users = proc_col(df.ID_User)
    idx2tempat, tempat_col, num_tempat = proc_col(df.ID_Tempat)
    df.ID_User = user_col
    df.ID_Tempat = tempat_col
    return df, idx2user, idx2tempat, num_users, num_tempat

In [32]:
df, idx2user, idx2tempat, num_users, num_tempat = encode_data(df)

## CREATING EMBEDDINGS

In [33]:
def create_embedings(n, num_factors):
    embedding = 6 * np.random.random((n, num_factors)) / num_factors
    return embedding

In [34]:
def df2matrix(df, nrows, ncols, column_name="Rating"):
    values = df[column_name].values
    ind_tempat = df['ID_Tempat'].values
    ind_user = df['ID_User'].values
    return sparse.csc_matrix((values, (ind_user, ind_tempat)), shape=(nrows, ncols))

In [35]:
Y = df2matrix(df, num_users, num_tempat)

In [36]:
def sparse_multiply(df, emb_user, emb_tempat):
    df["prediction"] = np.sum(emb_user[df["ID_User"].values] * emb_tempat[df["ID_Tempat"].values],axis=1)
    return df2matrix(df, emb_user.shape[0], emb_tempat.shape[0], column_name="prediction")

In [37]:
def cost(df, emb_user, emb_tempat):
    df["prediction"] = np.sum(emb_user[df["ID_User"].values]*emb_tempat[df["ID_Tempat"].values], axis=1)
    error = np.mean(np.square(df.prediction - df.Rating))
    return error

In [38]:
def gradient(df, Y, emb_user, emb_tempat):
    R = Y.sign().todense()
    delta = np.multiply(Y.todense(),R) - sparse_multiply(df,emb_user,emb_tempat).todense()
    d_emb_user = -2*np.dot(delta,emb_tempat)/len(Y.data)
    d_emb_tempat = -2*np.dot(delta.transpose(),emb_user)/len(Y.data)
    return d_emb_user,d_emb_tempat

In [39]:
def gradient_descent(df,
                     emb_user,
                     emb_tempat,
                     iterations=100,
                     learning_rate=0.01,
                     df_val=None):
    Y = df2matrix(df, emb_user.shape[0], emb_tempat.shape[0])
    grad_u_moment, grad_m_moment = gradient(df, Y, emb_user, emb_tempat)
    emb_user = np.array(np.subtract(emb_user, learning_rate * grad_u_moment))
    emb_tempat = np.array(np.subtract(emb_tempat, learning_rate * grad_m_moment))
    for i in range(iterations - 1):
        grad_user, grad_tempat = gradient(df, Y, emb_user, emb_tempat)
        grad_u_moment = .9 * grad_u_moment + .1 * grad_user
        grad_m_moment = .9 * grad_m_moment + .1 * grad_tempat
        emb_user = np.array(
            np.subtract(emb_user, learning_rate * grad_u_moment))
        emb_tempat = np.array(
            np.subtract(emb_tempat, learning_rate * grad_m_moment))
        if i % 50 == 0:
            print("Training cost:", cost(df, emb_user, emb_tempat))
        if df_val is not None and i % 50 == 0:
            print("Validation cost:", cost(df_val, emb_user, emb_tempat))
    return emb_user, emb_tempat

In [40]:
K = 50
emb_user = create_embedings(num_users, K)
emb_tempat = create_embedings(num_tempat, K)

In [41]:
msk = np.random.rand(len(df)) < 0.8
train = df[msk].copy()
val = df[~msk].copy()

In [42]:
train.shape

(630, 3)

In [43]:
val.shape

(182, 3)

In [44]:
emb_user, emb_tempat = gradient_descent(train, emb_user, emb_tempat, iterations=500, learning_rate=1,df_val=val)

Training cost: 18.56783325177286
Validation cost: 19.60052528611508
Training cost: 2.216374317378918
Validation cost: 12.28362056602548
Training cost: 0.08611435795844075
Validation cost: 10.552582889377485
Training cost: 0.0119708494119702
Validation cost: 10.553765140670505
Training cost: 0.004689400336291336
Validation cost: 10.555254570347891
Training cost: 0.002322703302049837
Validation cost: 10.557362914592188
Training cost: 0.0012358250315022943
Validation cost: 10.55915931807908
Training cost: 0.0006751511084151761
Validation cost: 10.560688963322155
Training cost: 0.00037548945038722855
Validation cost: 10.561916442272164
Training cost: 0.00021356248332966314
Validation cost: 10.562865760011425


In [45]:
emb_tempat.shape

(100, 50)

In [46]:
emb_user.shape

(780, 50)

In [47]:
val.head()

,ID_User,ID_Tempat,Rating,prediction
7,6,0,5,1.381551
14,13,1,5,1.964728
25,24,2,5,1.468584
26,25,2,5,1.481920
27,26,2,5,1.301355
